In [ ]:
from planet4 import markings
import hdbscan
plot_kwds = {'alpha' : 0.8, 's' : 80, 'linewidths':0}

In [ ]:
import seaborn as sns
%matplotlib nbagg
sns.set_context('poster')
sns.set_style('white')
sns.set_color_codes()

In [ ]:
imid = markings.ImageID('b89', scope='planet4')

In [ ]:
blotches = imid.get_blotches()

In [ ]:
fans = imid.get_fans()

In [ ]:
imid.plot_fans()

In [ ]:
X = fans['x y'.split()]

In [ ]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=3, gen_min_span_tree=True)

In [ ]:
clusterer.fit(X.as_matrix())

In [ ]:
fig, ax = plt.subplots(ncols=2)
# plt.figure()
imid.plot_fans(img=True,ax=ax[0]);
imid.plot_fans(img=False,ax=ax[1], user_color='w');
clusterer.minimum_spanning_tree_.plot(axis=ax[1],edge_cmap='viridis', 
                                      edge_alpha=0.6, 
                                      node_size=80, 
                                      edge_linewidth=2);


In [ ]:
plt.figure()
clusterer.single_linkage_tree_.plot(cmap='viridis', colorbar=True)

In [ ]:
plt.figure()
clusterer.condensed_tree_.plot()

In [ ]:
plt.figure()
clusterer.condensed_tree_.plot(select_clusters=True, selection_palette=sns.color_palette())

In [ ]:
palette = sns.color_palette(n_colors=len(clusterer.labels_)-1)
cluster_colors = [sns.desaturate(palette[col], sat) 
                  if (col >= 0 and sat > 0.65) else (1.0, 1.0, 1.0) for col, sat in 
                  zip(clusterer.labels_, clusterer.probabilities_)]
test_data = X.as_matrix()
fig, ax = plt.subplots()
imid.show_subframe(ax=ax)
ax.scatter(test_data.T[0], test_data.T[1], c=cluster_colors, **plot_kwds)